In [1]:
# import libraries
import wandb
import pandas as pd
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
run = wandb.init(project="usa_census", group="eda", save_code=True)
local_path = wandb.use_artifact("census.csv:latest").file()
df = pd.read_csv(local_path, " ")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: milon101. Use `wandb login --relogin` to force relogin


/home/imdadul/miniconda3/envs/nyc_airbnb_dev/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3398: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
df = pd.read_csv(local_path, skipinitialspace=True)

In [2]:
run.finish()

In [5]:
# using pandas_profilisng library to get an overall overview of the data
import pandas_profiling
                                                                                                                                                                                                                    
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/29 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
df["native-country"].unique()
# df.columns

array(['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico',
       'South', 'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany',
       'Iran', 'Philippines', 'Italy', 'Poland', 'Columbia', 'Cambodia',
       'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
       'China', 'Japan', 'Yugoslavia', 'Peru',
       'Outlying-US(Guam-USVI-etc)', 'Scotland', 'Trinadad&Tobago',
       'Greece', 'Nicaragua', 'Vietnam', 'Hong', 'Ireland', 'Hungary',
       'Holand-Netherlands'], dtype=object)

 There are missing values in a few columns and the column `last_review` is a date but it is in string format. Look also at the `price` column, and note the outliers. There are some zeros and   some very high prices. After talking to your stakeholders, you decide to consider from a minimum of \$ 10 to a maximum of \$ 350 per night. Fix some of the little problems we have found in the data with the following code:

In [2]:

import numpy as np
df = df.replace('\?', np.nan, regex=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              30162 non-null  int64 
 1    workclass       30162 non-null  object
 2    fnlwgt          30162 non-null  int64 
 3    education       30162 non-null  object
 4    education-num   30162 non-null  int64 
 5    marital-status  30162 non-null  object
 6    occupation      30162 non-null  object
 7    relationship    30162 non-null  object
 8    race            30162 non-null  object
 9    sex             30162 non-null  object
 10   capital-gain    30162 non-null  int64 
 11   capital-loss    30162 non-null  int64 
 12   hours-per-week  30162 non-null  int64 
 13   native-country  30162 non-null  object
 14   salary          30162 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [3]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()


Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
cat_features = [
        " workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country"
    ]

df[cat_features[0]].unique()

for cls in df[cat_features[0]].unique():
            df_temp = df[df[cat_features[0]] == cls]
            print(df_temp.head())

     age   workclass   fnlwgt      education   education-num  \
0     39   State-gov    77516      Bachelors              13   
11    30   State-gov   141297      Bachelors              13   
34    22   State-gov   311512   Some-college              10   
48    41   State-gov   101603      Assoc-voc              11   
123   29   State-gov   267989      Bachelors              13   

          marital-status       occupation    relationship  \
0          Never-married     Adm-clerical   Not-in-family   
11    Married-civ-spouse   Prof-specialty         Husband   
34    Married-civ-spouse    Other-service         Husband   
48    Married-civ-spouse     Craft-repair         Husband   
123   Married-civ-spouse   Prof-specialty         Husband   

                    race    sex   capital-gain   capital-loss  \
0                  White   Male           2174              0   
11    Asian-Pac-Islander   Male              0              0   
34                 Black   Male              0       

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.


In [ ]:
# drop outliers
min_price = 10
max_price = 340
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# convert last review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])